![A soccer pitch for an international match.](soccer-pitch.jpg)

You're working as a sports journalist at a major online sports media company, specializing in soccer analysis and reporting. You've been watching both men's and women's international soccer matches for a number of years, and your gut instinct tells you that more goals are scored in women's international football matches than men's. This would make an interesting investigative article that your subscribers are bound to love, but you'll need to perform a valid statistical hypothesis test to be sure!

While scoping this project, you acknowledge that the sport has changed a lot over the years, and performances likely vary a lot depending on the tournament, so you decide to limit the data used in the analysis to only official `FIFA World Cup` matches (not including qualifiers) since `2002-01-01`.

You create two datasets containing the results of every official men's and women's international football match since the 19th century, which you scraped from a reliable online source. This data is stored in two CSV files: `women_results.csv` and `men_results.csv`.

The question you are trying to determine the answer to is:

> Are more goals scored in women's international soccer matches than men's?

You assume a **10% significance level**, and use the following null and alternative hypotheses:

$H_0$ : The mean number of goals scored in women's international soccer matches is the same as men's.

$H_A$ : The mean number of goals scored in women's international soccer matches is greater than men's.

In [9]:
# Start your code here!
import pandas as pd
import pingouin 
from scipy.stats import mannwhitneyu

women_results = pd.read_csv('women_results.csv');
men_results = pd.read_csv('men_results.csv');

women_tournament_count = women_results['tournament'].value_counts()
men_tournament_count = men_results['tournament'].value_counts()

women_results['date'] = pd.to_datetime(women_results['date'])

womenFilteredDate = women_results[(women_results['tournament'].isin(['FIFA World Cup'])) & (women_results['date'] > '2002-01-01')]

men_results['date'] = pd.to_datetime(men_results['date'])
menFilteredDate = men_results[(men_results['tournament'].isin(['FIFA World Cup'])) & (men_results['date'] > '2002-01-01')]

womenFilteredDate['group'] = "women"
menFilteredDate['group'] = "men"

womenFilteredDate['goalScored'] = womenFilteredDate['home_score'] + womenFilteredDate['away_score']
menFilteredDate['goalScored'] = menFilteredDate['home_score'] + menFilteredDate['away_score']

womenAndMenTeam = pd.concat([womenFilteredDate,menFilteredDate], axis=0,ignore_index=True)

womenAndMenGroupAndScore = womenAndMenTeam[['group','goalScored']]
womenAndMenGroupAndScorePivot = womenAndMenGroupAndScore.pivot(columns='group', values="goalScored")

resultWithPg = pingouin.mwu(x=womenAndMenGroupAndScorePivot['women'],y=womenAndMenGroupAndScorePivot['men'],alternative="greater")

resultWithScipy = mannwhitneyu(x=womenFilteredDate['goalScored'],y=menFilteredDate['goalScored'],alternative="greater")

p_val = round(resultWithPg['p-val'].values[0],4)

if p_val <= 0.01:
    result = "reject"
else:
    result ="fail to reject"
    
result_dict = {"p_val": p_val, "result": result}

print(result_dict)
print(p_val)
print(resultWithScipy)
print(resultWithPg)
print(womenAndMenGroupAndScorePivot)
print(womenAndMenGroupAndScore)
print(womenAndMenTeam)
print(menFilteredDate)





{'p_val': 0.0051, 'result': 'reject'}
0.0051
MannwhitneyuResult(statistic=43273.0, pvalue=0.005106609825443641)
       U-val alternative     p-val       RBC      CLES
MWU  43273.0     greater  0.005107 -0.126901  0.563451
group  men  women
0      NaN    3.0
1      NaN    2.0
2      NaN    5.0
3      NaN    6.0
4      NaN    4.0
..     ...    ...
579    3.0    NaN
580    3.0    NaN
581    2.0    NaN
582    3.0    NaN
583    6.0    NaN

[584 rows x 2 columns]
     group  goalScored
0    women           3
1    women           2
2    women           5
3    women           6
4    women           4
..     ...         ...
579    men           3
580    men           3
581    men           2
582    men           3
583    men           6

[584 rows x 2 columns]
     Unnamed: 0       date      home_team  ...      tournament  group  goalScored
0          1600 2003-09-20        Nigeria  ...  FIFA World Cup  women           3
1          1601 2003-09-20         Norway  ...  FIFA World Cup  women     